In [10]:
import pandas as pd
import sys
from data_reader import fes_reader, perf_reader, well_renaming, rename_columns
from tqdm import tqdm
import bisect
import json
import datetime

In [2]:
# конвертация путей файлов в зависимости от системы
def replace_slash(file_path):
    platform = sys.platform
    slash_map = {'win32': '\\',
                'cygwin': '\\',
                'darwin': '/',
                'linux2': '/'}
    if platform not in slash_map.keys(): platform = 'linux2'
    return file_path.replace('\\', slash_map[platform])

In [3]:
perf_path1 = replace_slash('input_data\\' + 'armitz/miner/ARX_PERF.xlsx')
fes_path1 = replace_slash('input_data\\armitz/miner/QINF.xlsx')

perf_path = replace_slash('input_data\\' + 'Перфорация.json')
fes_path = replace_slash('input_data\\РИГИС.json')

In [4]:
with open(fes_path, 'r') as f:
    data = json.dumps(json.load(f))
with open(perf_path, 'r') as f:
    data_perf = json.dumps(json.load(f))

In [5]:
perf_df = pd.read_json(data_perf, orient='records')

fes_df = pd.read_json(data, orient='records')

In [7]:
perf_df1 = pd.read_excel(perf_path1, engine='openpyxl', skiprows=1)
fes_df1 = pd.read_excel(fes_path1, engine='openpyxl', skiprows=1)

In [ ]:
perf_df.rename(columns=lambda x: x.lower().strip(), inplace=True)
rename_columns(perf_df)

perf_df1.rename(columns=lambda x: x.lower().strip(), inplace=True)
rename_columns(perf_df1)

fes_df.rename(columns=lambda x: x.lower().strip(), inplace=True)
rename_columns(fes_df)

fes_df1.rename(columns=lambda x: x.lower().strip(), inplace=True)
rename_columns(fes_df1)

In [12]:
perf_df['well'] = perf_df['well'].apply(well_renaming)
perf_df1['well'] = perf_df1['well'].apply(well_renaming)
fes_df['well'] = fes_df['well'].apply(well_renaming)
fes_df1['well'] = fes_df1['well'].apply(well_renaming)


In [10]:
fes_df1

,Цех,Скваж,Ствол,Зона,Профиль,Дата_бур,Мест,Площ,Гориз,Тек_заб,...,Н_нас,Характер_насыщения,Литология,Абс_кр,Абс_под,Мах_зен_уг,Глуб_мах_уг,Мin_зен_уг,D_эк,Н_цем
0,8,1,1.0,NaN,HНС,1950-02-22,1.0,905.0,531.0,NaN,...,NaN,NaN,не коллек,NaN,NaN,4.30,1000.0,0.1,168.0,NaN
1,8,1,1.0,NaN,HНС,NaT,1.0,905.0,531.0,NaN,...,69.5,NaN,алевролит,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8,1,1.0,NaN,HНС,NaT,1.0,905.0,531.0,NaN,...,NaN,NaN,не коллек,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8,1,1.0,NaN,HНС,NaT,1.0,905.0,531.0,NaN,...,NaN,NaN,не коллек,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8,1,1.0,NaN,HНС,NaT,1.0,905.0,531.0,NaN,...,NaN,NaN,не коллек,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156269,9,32960,1.0,1.0,ОРЭ,2010-03-26,NaN,NaN,NaN,1926.0,...,52.9,нефть слабо,загл.песч.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156270,9,32960,1.0,1.0,ОРЭ,NaT,NaN,NaN,NaN,NaN,...,NaN,коллектор обводненны,песчаник,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156271,9,32960,1.0,NaN,ОРЭ,2010-03-26,1.0,908.0,531.0,1926.0,...,NaN,верхний известняк,не коллек,1478.6,1480.0,32.96,1300.0,7.8,146.0,68.0
156272,9,32960,1.0,2.0,ОРЭ,2010-03-26,NaN,NaN,NaN,1926.0,...,NaN,коллектор обводненны,загл.песч.,1482.1,1483.5,NaN,NaN,NaN,NaN,NaN


In [4]:
perf_ints = perf_reader(perf_path)
fes_dict = fes_reader(fes_path)
SOIL_CUT = 60

started reading perf xl
done reading perf xl
started reading fes xl
done reading fes xl


In [14]:
fes_df1

,well,soil
0,1,NaN
1,1,69.5
2,1,NaN
3,1,NaN
4,1,NaN
...,...,...
156269,32960,52.9
156270,32960,NaN
156271,32960,NaN
156272,32960,NaN
